In [169]:
# импорты
import nltk
import re
import pickle
from string import punctuation
import math
from tqdm import tqdm_notebook
import csv
import gensim
import random
from collections import Counter
from nltk.util import ngrams

import pymystem3
m = pymystem3.Mystem() #для использования лемматизации

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

import pandas as pd
import numpy as np

'''from nltk.corpus import stopwords
stop_words = stopwords.words("russian")'''

'from nltk.corpus import stopwords\nstop_words = stopwords.words("russian")'

In [5]:
# читает из файла, убирает двойные пробелы и ручные переносы, последний \n
def reading(file):
    f = open('texts\\{}.txt'.format(file), 'r', encoding='utf-8')
    text = f.read()
    text = text.replace('  ', ' ')
    text = text.replace('-\n', '')
    if text[-1] == '\n': # убираем последний \n, если такой есть 
        text = text[:-1]
    f.close()
    return text

## Загрузка необходимых лексических баз:   
### словарей частот

In [ ]:
def unpack(data):
    input = open(data, 'rb')
    obj = pickle.load(input)
    input.close()
    return obj

In [2]:
# загружаем частоты лем униграмм
unigrams = unpack('1stemgrams.data')
# убрали пробел из начала слов
unigrams = {w[1:]:f for w,f in unigrams.items()}

In [3]:
bigrams = unpack('2grams.data')
# убрали пробел из начала слов, пунктуацию (кроме дефисов) и двойные пробелы
bigrams  = {''.join([i for i in w[1:] if i not in punctuation.replace('-','')]).replace('  ',' '):f for w,f in bigrams.items()}

In [4]:
list(bigrams.keys())[:10]

['дальнейший допрос',
 'оканчивать преступление',
 'развитие христианство',
 'и затонуть',
 'подразумевать она',
 'и однозначный',
 '2 введение',
 'желать услышать',
 'являться недооценка',
 'закрепляться по']

In [4]:
trigrams = unpack('3grams.data')
# убрали пробел из начала слов, пунктуацию (кроме дефисов) и двойные пробелы
trigrams = {''.join([i for i in w[1:] if i not in punctuation.replace('-','')]).replace('  ',' '):f for w,f in trigrams.items()}

In [6]:
list(trigrams.keys())[:10]

['в лифт я',
 'что сам русский',
 'почему ты думать',
 'фестиваль в кольмар',
 'твердость и прочность',
 'тот число руководитель',
 'судьба решать иначе',
 'мысль рождаться в',
 'мой учение то',
 'да только что']

## Загрузка необходимых лексических баз:   
### модели

In [16]:
# загрузка модели
def model_loading(file):
    model = gensim.models.KeyedVectors.load_word2vec_format(file, binary=False)
    model.init_sims(replace=True)
    print('Done!') 
    return model

model = model_loading('news_upos_cbow_600_2_2018.vec') #на загрузку тратится минут 10

Done!


## Загрузка необходимых лексических баз:   
### словаря

In [17]:
# загрузка словаря ASIS
with open('syns.data', 'rb') as f:
     asis = pickle.load(f)

## Загрузка необходимых лексических баз:   
### минимумов

In [23]:
def load_minimum(typ=['A1','B1','B2'][0]):
    # всякие непонятные куски, портящие статистику по минимумам:
    to_remove = ['ми', 'вод', 'кий', 'ре','вая', 'ча', 'то быть', 'чка', 'стен']
    with open('{}.txt'.format(typ), 'r', encoding='utf-8') as file:
        lemtokens = [morph.parse(i)[0].normal_form for i in file.read().lower().split('\n') if morph.parse(i)[0].normal_form not in to_remove] #делаем список лемм слов
        minimum = list(set(lemtokens)) #убираем повторы\
    return minimum

In [174]:
global_threshold

-9.6

In [82]:
minimum = load_minimum('B2')

## Задание параметров способа определения сложности (частота или информативность), идентификации слова как сложного (основываться только на минимуме/нет) и порога сложности

In [28]:
# ПАРАМЕТРЫ анализа слов
complexity_type = 'inf' # альтернатива: freq 
use_min=False

### Определение порога сложности на основе коэффициента информации (из униграмм) и минимумов

In [26]:
def make_threshold():
    # чем коэффициент информативности меньше, чем сложнее слово (с уменьшением дроби логарифм уменьшается тоже)
    det = sum(f for f in unigrams.values())+1
    # словарь коэффициентов информативности и количества слов с таким коэф. в выбранном минимуме
    inf_coef = Counter([round(math.log((unigrams.get(w)+1)/det),1) for w in minimum if w in unigrams])
    # может быть очень низким (<17) для очень редких слов (матерь??) или опечаток, но в основном колеблется ок -9
    print(inf_coef.most_common(10))
    return np.median(list(inf_coef.keys()))

In [27]:
global_threshold = make_threshold()

[(-9.3, 68), (-9.4, 49), (-9.8, 47), (-9.7, 46), (-8.9, 45), (-9.6, 45), (-9.5, 44), (-10.2, 40), (-9.2, 37), (-9.0, 37)]


## Таблица конверсии в UPoS из тэгов Mystem: в моделях используется первый, а при анализе текстов второй. Таблица конверсии из тегов Pymorphy2 - используется при назначении части речи словарным словам

In [18]:
# Таблица конверсии в UPoS из тэгов Mystem
# словарь, переводящий теги mystem в universal теги моделей
mystem_tags = {'A' : 'ADJ',
       'ADV' : 'ADV',
       'ADVPRO' : 'ADV',
       'ANUM' : 'ADJ',
       'APRO' : 'DET',
       'COM' : 'ADJ',
       'CONJ' : 'SCONJ',
       'INTJ' : 'INTJ',
       'NONLEX' : 'X',
       'NUM' : 'NUM',
       'PART' : 'PART',
       'PR' : 'ADP',
       'S' : 'NOUN',
       'SPRO' : 'PRON',
       'UNKN' : 'X',
       'V' : 'VERB',
       'X' : 'X',
      'PROPN' : 'PROPN'} #последних 2 тегов в майстеме нет, но они задаются в классе для слов в соответсвующими пометами

# словарь, переводящий теги пайморфи в universal теги моделей
pymorphy_tags = {'ADJF':'ADJ',
    'ADJS' : 'ADJ',
    'ADVB' : 'ADV',
    'COMP' : 'ADV',
    'GRND' : 'VERB',
    'INFN' : 'VERB',
    'NOUN' : 'NOUN',
    'PRED' : 'ADV',
    'PRTF' : 'ADJ',
    'PRTS' : 'VERB',
    'VERB' : 'VERB'}

## Основные классы: Токен (родительский), Сложное слово и Замена (дочерние)

In [29]:
class Token():
    def __init__(self, w):
        
        self.num = None # номер в тексте
        self.complexity = None # сложность слова
        self.av_similarity = None
        
        # три варианта инициализации: 
        ## из анализа текста, 
        ## из уже имеющегося объекта (для дочернего класса ComplexWord) 
        ## из строки
        
        if isinstance(w, dict): # если получили результат работы mystem
            
            self.text = w['text']  # сам токен
            self.len = len(w['text']) # его длина
            
            # определяет, сделан ли анализ и, соответственно, рассматривать ли как слово, требующее упрощения
            gram = w.get('analysis')
            if gram:
                self.lexem = gram[0]['lex']  # лемма
                
                if not self.named_entity(gram[0]):  # именованная сущность или нет
                    self.pos = self.pos_tag(gram[0]['gr'])  # часть речи
                else:
                    self.pos = 'PROPN' # universal tag for named entity - у майстема таких нет
                
                    
            elif any(p in w['text'] for p in punctuation+'–«»'): # если это знак пунктуации (может быть с пробелом!)
                self.lexem = '_PUNKTUATION_'
                self.pos = None
            
            elif not re.findall('\S',w['text']): # если это только пробельные символы
                self.lexem = '_SPACE_'
                self.pos = None
                
            # остальное - неизвестная и ненужная ерунда?
            else:
                self.lexem = '_UNK_'
                self.pos = 'X' # universal tag for unknown
            
            
        elif isinstance(w, Token): # для определения объектов дочернего класса ComplexWord
            self.text = w.text
            self.num = w.num
            self.lexem = w.lexem 
            self.len = w.len
            self.pos = w.pos
            self.complexity = w.complexity
            
        
        elif isinstance(w, str): # если хотим как класс токен определить строку, полученную из словаря или модели
            self.text = w
            self.pos = None
            self.lexem = w
            self.len = len(w)
            self.num = None
            self.complexity = None
            
        
    # вытаскивает часть речи из разбора майстем
    def pos_tag(self,gram):
        if ',' in gram:
            gram = gram.split(',')[0]
        if '=' in gram:
            gram = gram.split('=')[0]
        return gram
        
    # определяет по тегам, является ли именованной сущностью
    def named_entity(self,gram):
        markers = {'сокр': ' - сокращение', 'фам': ' - фамилия', 'имя': ' - имя собственное', 'гео': ' - название места', }
        if any(m in gram['gr'] for m in markers.keys()):
            return True
        else:
            return False

    def complexity_params(self, param = 'freq'):
        # если по частотности
        if param == 'freq':
            self.complexity = unigrams.get(self.lexem, 0)
            
        # если по коэффициенту информативности. Отрицательное значение. Чем он меньше, тем сложнее
        elif param == 'inf':
            self.complexity = math.log((unigrams.get(self.lexem, 0)+1)/(sum(f for f in unigrams.values())+1))
        
    
    def is_complex(self, threshold = global_threshold, use_min = False, len_threshold = 1000):
        exceptions = ['_PUNKTUATION_', '_SPACE_', '_UNK_']
        # проверка, что это слово и что его нужно рассматривать как сложное (не нарицательное)
        if not any(exception in self.lexem for exception in exceptions) and self.pos not in ['PROPN']:
            
            # если показатель сложности - вхождение в минимум
            if use_min:
                if self.lexem not in minimum:
                    return True
                else:
                    return False

            # если показатель сложности - пороговое значение сложности
            # также может использоваться длина. По умолчанию слишком большое - 1000 (т.е. этот параметр не учитывается)
            else:
                if self.complexity < threshold or self.len > len_threshold:
                    return True
                else:
                    return False
        else:
            return False
    
    def convert_universal(self):
        if self.pos in mystem_tags:
            self.pos = mystem_tags[self.pos]
        else:
            self.pos = 'X' # Х - universal тег для неизвестных слов
        return self
    
    # нужна в двух случаях: для Замен, полученных из словарей, чтобы приписывать им соответствующий параметр,
    # и для подсчета адекватности слова контексту
    def cos_sim(self, context):
        #print(self.lexem, target.lexem, self.pos, target.pos)
        subst_query = str(self.lexem+'_'+self.pos)
        target_query = str(context.lexem+'_'+context.pos)
        if subst_query in model and target_query in model:
            return model.similarity(subst_query, target_query)
        else:
            return 0.0

In [53]:
class Substitution(Token):
    def __init__(self, w):
        super().__init__(w)
        self.similarity = None
        self.fitness = None
        self.closeness = None
    
    # для слов из словаря и тезауруса: определяем тег пайморфи, переводим в формат universal - так быстрее, чем майстемом
    def tagging(self, w):
        tag = morph.parse(w)[0].tag.POS
        if tag in pymorphy_tags:
            return pymorphy_tags[tag]
        else:
            return 'X' # Х - universal тег для неизвестных слов
    
    # приписываем недостающие атрибуты словам, взятым из словаря или тезауруса
    def setting_atr(self, target):
        self.pos = self.tagging(self.lexem)
        self.complexity_params(complexity_type)
        self.similarity = self.cos_sim(target)

In [54]:
class Complex_word(Token):
    def __init__(self, w):
        super().__init__(w)
        self.substituts = None
        self.place = None
        self.context = None
        self.easier = []
        
    # список замен, в зависимости от выбранной базы
    def search_substituts(self, base_type='model'):
        
        # поиск по модели
        def model_search(lexem, pos):
            query = str(lexem+'_'+pos)
            #print(query)
            if query in model:
                # формируем список квазисинонимов той же части речи
                # при этом превращаем их в объекты соответствующего класса
                syn_tokens = []
                for syn, sim in model.most_similar(positive=[query]):
                    syn_text = syn[:syn.find('_')] # текст до части речи
                    
                    syn_tok = Substitution(syn_text) # из текстовой строки инициализируем объект класса
                    syn_tok.pos = syn[syn_tok.len+1:] # часть речи
                    
                    syn_tok.complexity_params(complexity_type) # сложность по функции в зависимости от выбранного параметра
                    
                    syn_tok.similarity = sim # а близость по параметру модели
                    
                    syn_tokens.append(syn_tok)
                    
                return syn_tokens
            else:
                return [] 

        # поиск по YARN
        def yarn_search(target, filepath = 'yarn-synsets1.csv'):
            with open(filepath, "r", newline="") as file: # постепенный просмотр файла с синсетами (множествами синонимов)
                reader = csv.DictReader(file, delimiter=';')
                lst = []
                for i,row in enumerate(reader):
                    cur_line = row['words'].split(';') # считываем колонку с синсетами
                    if len(cur_line)>1:
                        if target.lexem in cur_line:
                            del(reader)
                            for c in cur_line:
                                if ' ' not in c and c!=target.lexem: # формируем список однословных синонимов
                                    sub_tok = Substitution(c)
                                    
                                    sub_tok.setting_atr(target)
                                    
                                    if sub_tok not in lst:
                                        lst.append(sub_tok) 
                            #TODO: выделить неоднословные в отдельный класс и поискать их частотность по n-граммам?
                            break
                #print(lst)
                return lst 
        
        #поиск по ASIS
        def asis_search(target):
            if target.lexem in asis:
                lst = []
                for s in asis[target.lexem]:
                    if ' ' not in s: # формируем список однословных синонимов
                        sub_tok = Substitution(s)
                        sub_tok.setting_atr(target)
                        lst.append(sub_tok)
                return lst
            else:
                return []

        
        if base_type == 'model':
            self.substituts = model_search(self.lexem, self.pos)
            
        if base_type == 'yarn':
            self.substituts = yarn_search(self)
        
        if base_type == 'asis':
            self.substituts = asis_search(self)
            
        return self
    
    def find_easier(self, use_min = False, threshold = global_threshold):
        for sub in self.substituts:
            if not sub.is_complex(threshold = threshold, use_min = use_min):
                if sub.complexity > self.complexity:
                    self.easier.append(sub)
        return self
    
    def make_window(self, tokens, window = 10):
        context = [self]
        left_ind = self.num-1
        right_ind = self.num+1
        ind = 0
        # добавляем по одному слову слева и/или справа, пока не наберется window + само слово
        while len(context)<window+1:
            while left_ind >= 0:
                left_w = tokens[left_ind]
                left_ind-=1
                # проверка, что это слово
                if any(exception in left_w.lexem for exception in ['_PUNKTUATION_', '_SPACE_', '_UNK_']):
                    continue
                else:
                    context[:0] = [left_w] #вставляем слово слева от цепочки
                    ind+=1 # индекс слова сдвигается
                    break

            while right_ind < len(tokens):
                right_w = tokens[right_ind]
                right_ind+=1
                # проверка, что это слово и что его нужно рассматривать как сложное (не нарицательное)
                if any(exception in right_w.lexem for exception in ['_PUNKTUATION_', '_SPACE_', '_UNK_']):
                    continue
                else:
                    context.append(right_w) # справа от цепочки
                    break
        self.place = ind
        self.context = context
        return self

## Функция анализа текста и формирования из него списка токенов с соотв.атрибутами. Вход: текст в виде строки

In [55]:
#анализ текста 
def text_structuring(text, param, threshold, use_min):
    # анализирует текст 
    analysis = m.analyze(text)
    tokens = []
    for i, w in enumerate(analysis): # состаляем список объектов Tokens
        token = Token(w)
        token.num = i # добавляем токену в атрибуты его номер в тексте
        token.complexity_params(param) # переопределяем сложность на основе выбранного параметра
        token.convert_universal() # превращаем POS в universal формат
        #print(token.num, token.text, token.lexem, token.pos, token.complexity, token.is_complex(threshold, use_min))
        tokens.append(token)
    return tokens

## Функция для нахождения сложных слов, превращения их в подкласс сложных слов и поиск замен для них. Несложные слова также остаются на своих местах. Вход: список объектов класса Токен

In [127]:
def selecting_complex(tokens, base_type=['yarn','model','asis'][0], threshold = global_threshold, use_min = False):
    complex_words = []
    for token in tokens:
        if token.is_complex(threshold, use_min):
            comp_token = Complex_word(token) # токен становится Сложным словом
            
            comp_token.search_substituts(base_type=base_type)
            
            complex_words.append(comp_token)
            
            # код для принтов
            '''
            if comp_token.substituts:
                for syn in comp_token.substituts:
                    print (comp_token.lexem, comp_token.complexity, base_type, ':', syn.lexem, syn.complexity, syn.similarity)
            '''
            if comp_token.substituts:
                comp_token.find_easier(threshold, use_min)
            '''if comp_token.easier:
                for syn in comp_token.easier:
                    print (comp_token.lexem, comp_token.pos, base_type, ':', syn.lexem, syn.pos, syn.complexity, syn.similarity)
            '''
        else:
            complex_words.append(token)
    return complex_words

## Вероятность контекстного окна на основе логарифма + Лапласса. Контекстное окно: 2n+1 слов (по n слева и справа), можно задавать. 

In [57]:
ngrams_dict = {0: {}, 1: unigrams, 2: bigrams, 3: trigrams}

### Вообще при н=1 должно быть не деление на V, а деление на объем корпуса - количество вхождений слов. Те подсчитать сумму значений частот униграмм. Но пока для 3грамм работает нормально. Также криво, вроде бы, с логарифмом... 

In [155]:
def ngram_prob(ngram, n, use_log= False):
    c1 = ' '.join(ngram[-n:]) # в числителе частота строки длины n
    c2 = ' '.join(ngram[-n:-1]) # в знаменателе - строки без последнего символа
    d = ngrams_dict.get(n) # для поиска числителя берем словарь n-грамм
    d2 = ngrams_dict.get(n-1) # для поиска знаменателя - словарь n-1-грамм
    V = len(d)
    #len(ngrams_dict.get(n-1,len(unigrams))) # сглаживание лапласса: добавляем размер словаря знаменателя
    p1 = d.get(c1,0)+1
    p2 = d2.get(c2,0)+V
    
    result =  p1/p2
    
    #print('\t',c1, '/', c2, ':' , p1-1, '/', p2-V, '(', p1, '/', p2, ')', '=', result)
    
    if use_log:
        return math.log(p1)-math.log(p2)
    else:
        return p1/p2
    
def context_prob(ngrams, use_log= False):
    if use_log:
        p_context = 0.0
    else:
        p_context = 1.0
    for ngram in ngrams:
        p = ngram_prob(ngram, 3, use_log= use_log)
        #print(ngram, p)
        if use_log:
            p_context+=p
        else:
            p_context*=p
    return p_context

## Загрузка датафрейма с текстами и измерение потенциала системы для разных словарных баз

In [117]:
df = pd.read_csv('texts1.csv', sep='\t', encoding='utf-8')

In [118]:
df.drop(['Unnamed: 0'],axis=1)

,headlines,texts,rubrics,links
0,Трамп назвал главу Нью-Йорка худшим мэром в США,Президент США Дональд Трамп назвал главу Нью-Й...,world,https://news.yandex.ru//story/Tramp_nazval_gla...
1,В Германии недовольные «Северным потоком — 2» ...,Протестующие против строительства «Северного п...,world,https://news.yandex.ru//story/V_Germanii_nedov...
2,США внесут Huawei в черный список,"Американский Минторг объявил, что внесёт крупн...",world,https://news.yandex.ru//story/SSHA_vnesut_Huaw...
3,Мадуро прокомментировал решение США запретить ...,"Президент Венесуэлы Николас Мадуро заявил, что...",world,https://news.yandex.ru//story/Maduro_prokommen...
4,Под завалами рухнувшего в Шанхае здания погибл...,Спасателям удалось обнаружить и вытащить из по...,world,https://news.yandex.ru//story/Pod_zavalami_ruk...
5,Появились тизерные изображения новой BMW 1-Series,Новый BMW 1-Series разработан на базе платформ...,world,https://news.yandex.ru//story/Poyavilis_tizern...
6,Philips перешла в Нидерландах на возобновляему...,Нидерландские предприятия Royal Philips NV пол...,world,https://news.yandex.ru//story/Philips_pereshla...
7,Виновницей крупнейшего в Калифорнии пожара наз...,Представители калифорнийского управления лесно...,world,https://news.yandex.ru//story/Vinovnicej_krupn...
8,Рождаемость в США достигла минимума,"По данным экспертов, снижение рождаемости набл...",world,https://news.yandex.ru//story/Rozhdaemost_v_SS...
9,В Турции анализ ДНК выявил новую тайну древней...,Международная группа генетиков и археологов из...,world,https://news.yandex.ru//story/V_Turcii_analiz_...


In [119]:
rubrics = list(set(df['rubrics']))
rubrics

['world',
 'culture',
 'science',
 'society',
 'incident',
 'sport',
 'computers',
 'business',
 'politics']

In [101]:
def make_sample(n):
    rubric = list(set(df['rubrics'])) # список рубрик
    text_rubrics = df.filter(items=['texts', 'rubrics']) # таблица текст-рубрика
    text_for_potential = []
    for r in rubric:
        rub_texts = []
        for l in range(len(text_rubrics)):
            if text_rubrics.loc[l][1]==r:
                rub_texts.append(text_rubrics.loc[l][0])
        text_for_potential.extend(random.sample(rub_texts,n)) # делаем подвыборку в н текстов
    return text_for_potential

In [96]:
def measure_potential(n = 5, base = ['yarn','model','asis'][0], use_min = False):
    
    text_for_potential = make_sample(n)
    #print(text_for_potential)
    
    positif = 0
    total1 = 0
    total2 = 0
    
    for text in text_for_potential:
        tokens = text_structuring(text, complexity_type, global_threshold, use_min)        
        complex_words = selecting_complex(tokens, base, threshold = global_threshold, use_min = False)
        actually_complex = [token for token in complex_words if isinstance(token, Complex_word)]
        '''for token in actually_complex:
            print(token.num, token.text, token.lexem, token.complexity, token.easier)'''
        
        comlex_easier = [token for token in actually_complex if token.easier]
        
        positif += len(comlex_easier)
        total1 += len(actually_complex)
        
        '''positif += sum([sum([1 for sub in token.easier if sub in minimum]) for token in comlex_easier])
        total1 += len(actually_complex)
        total2 += len(comlex_easier)
        
        
    pot1 = positif/total1 if total1 else 0
    pot2 = positif/total2 if total2 else 0
    
    return pot1, pot2'''
    pot1 = positif/total1 if total1 else 0
    
    return pot1

In [97]:
model_potential = measure_potential(n = 3, base = 'model')
model_potential

0.42248062015503873

In [98]:
asis_potential = measure_potential(n = 3, base = 'asis')
asis_potential_total

0.0

In [99]:
yarn_potential = measure_potential(n = 3, base = 'yarn')
yarn_potential_total

0.0

## Большая и страшная функция для сложных токенов в тексте, вычисляющая для них всяческие параметры

In [170]:
class_df = pd.DataFrame(columns=['лемма','индекс','контекст', 'информативность','словарность','близость','вероятность','логарифм_вероятности','контекстуальность','класс'])
class_df

,лемма,индекс,контекст,информативность,словарность,близость,вероятность,логарифм_вероятности,контекстуальность,класс


In [168]:
def making_frame(window=5, base = ['yarn','model','asis'][0], n=3, t=5, algorithm = ['ngram','random_three'][0], baseline=0):
    # алогритм классификации слова как сложного/простого
    global class_df
    
    def ngram_algorithm():
        fit='НЕТ'
        if p_changed_context>p_context:
            fit='ДА'
            if sub.fitness > best_fitness:
                return sub
            elif sub.fitness == best_fitness:
                if sub.complexity - best_sub.complexity > baseline: 
                    # принимаю условие, что вероятность должна быть больше исходной (baseline=0) 
                    # можно модифицировать на "вероятность не более чем на _ меньше исходной (baseline=-0.01, например)
                    return sub
                else:
                    return best_sub
            else:
                return best_sub
            
    def transpose(matr):
        res=[]
        n=len(matr)
        m=len(matr[0])
        for j in range(m):
            tmp=[]
            for i in range(n):
                tmp=tmp+[matr[i][j]]
            res=res+[tmp]
        return res
            
    for text in make_sample(t):
        print(text)
        tokens = text_structuring(text, complexity_type, global_threshold, use_min)        
        complex_words = selecting_complex(tokens, base, threshold = global_threshold, use_min = False)
        actually_complex = [token for token in complex_words if isinstance(token, Complex_word)] 
        pr = [token.lexem for token in actually_complex]
        print(pr)
        complex_easier = [token for token in actually_complex if token.easier]
        pr = [token.lexem for token in complex_easier]
        print(pr)
        for token in complex_easier:
            #[0'лемма',1'индекс',2'контекст',3'информативность',4'словарность',
            # 5'близость',6'вероятность',7'логарифм_вероятности',8'контекстуальность',9'класс']
            
            new_line = [[0]]*10 #будущая часть датафрейма

            cls = [0]*len(token.easier) # для таблички - классы
            in_min = [] # для таблички - словарность - в минимуме
            easier_lexems = [t.lexem for t in token.easier] # для таблички - лексемы замен 0
            new_line[0] = easier_lexems
            easier_inf = [t.complexity for t in token.easier] # для таблички - сложность замен 3
            new_line[3] = easier_inf
            easier_cos = [sub.cos_sim(token) for sub in token.easier] # для таблички - косинусная близость 5 
            new_line[5] = easier_cos
            

            # окна контекста строим для тех сложных слов, для которых есть варианты замен проще, чем исходное
            token.make_window(complex_words, window = window) # для подсчетов
            easier_places = [token.place]*len(token.easier) # для таблички - место исходного в контексте 1
            new_line[1] = easier_places

            context_lem = [c.lexem for c in token.context] # для подсчета вероятностей нграмм
            context_text = ' '.join([c.text.lower() for c in token.context]) # для таблички - контекст 2
            new_line[2] = [context_text]*len(token.easier)

            # определение, насколько слово близко с другими словами из контекста
            # cos_sim возвращает 0.0, если пара слов не найдена в модели
            closeness = np.mean([token.cos_sim(w) for w in token.context if w.lexem!=token.lexem])

            text_ngrams = [g for g in ngrams(context_lem, n)] # генерация n-грамм
            p_context = context_prob(text_ngrams, use_log=False) # вероятность контекста
            p_context_log = context_prob(text_ngrams, use_log=True) # вероятность контекста

            best_fitness = -1000  # 
            best_sub = None
            easier_prob = [] # для таблички - контексная вероятность 6
            easier_prob_log = [] # для таблички - log контексная вероятность 7
            easier_similarity = [] # для таблички - контексная близость 8
            for sub in token.easier:
                
                # контекст с новым словом
                sub_context = token.context[:token.place]+[sub]+token.context[token.place+1:]
                # все то же самое
                sub.closeness = np.mean([sub.cos_sim(w) for w in sub_context if w.lexem!=sub.lexem])
                easier_similarity.append(sub.closeness) # для таблички - контексная близость

                sub_context_lem = [t.lexem for t in sub_context]
                sub_ngrams = [g for g in ngrams(sub_context_lem, n)]
                p_changed_context = context_prob(sub_ngrams, use_log=False)
                easier_prob.append(p_changed_context-p_context) # для таблички - контексная вероятность
                
                p_changed_context2 = context_prob(sub_ngrams, use_log=True)
                sub.fitness = p_changed_context2
                easier_prob_log.append(p_changed_context2-p_context_log) # для таблички - log контексная вероятность

                if sub in minimum:
                    in_min.append(1) # словарность
                else:
                    in_min.append(0)

                if algorithm == 'ngram':
                    best_sub = ngram_algorithm()
                    if best_sub:
                        best_fitness = best_sub.fitness
            
            new_line[4] = in_min # словарность
            new_line[6] = easier_prob # вероятность
            new_line[7] = easier_prob_log
            new_line[8] = easier_similarity # контекстуальность
            
            if algorithm == 'ngram' and best_sub:
                for i,sub in enumerate(token.easier):
                    if sub.lexem == best_sub.lexem:
                        cls[i]=1
                        
            new_line[9] = cls # класс
            
            print(new_line)
            # запись в датафрейм
            new_line = transpose(new_line)
            for line in new_line:
                class_df.loc[len(class_df)]=line
            print(new_line)
            
            
            

In [171]:
making_frame(window=5, base = 'yarn', n=3, t=15, algorithm = 'ngram', baseline=0)

Пожарная служба американского города Оклахома спасает двух мойщиков окон, которые оказались заблокированными на подъемнике, когда он находился в верхней части небоскреба. Инцидент произошел утром по местному времени в 50-этажном здании в центре города.
['пожарный', 'оклахома', 'мойщик', 'заблокировать', 'подъемник', 'небоскреб', 'инцидент', 'этажный']
['подъемник', 'инцидент']
[['лифт'], [3], ['оказались заблокированными на подъемнике когда он находился'], [-10.858659349964656], [0], [0.5144726968767824], [-2.462136204451526e-35], [-6.465851045334148e-05], [-0.0030786052273014653], [0]]
[['лифт', 3, 'оказались заблокированными на подъемнике когда он находился', -10.858659349964656, 0, 0.5144726968767824, -2.462136204451526e-35, -6.465851045334148e-05, -0.0030786052273014653, 0]]
[['происшествие', 'случай', 'событие'], [3, 3, 3], ['верхней части небоскреба инцидент произошел утром по', 'верхней части небоскреба инцидент произошел утром по', 'верхней части небоскреба инцидент произошел у

[['значит'], [3], ['нью-йорка демократа билла де блазио заявившего о'], [-7.997805837004221], [0], [0.0], [0.0], [0.0], [0.0], [0]]
[['значит', 3, 'нью-йорка демократа билла де блазио заявившего о', -7.997805837004221, 0, 0.0, 0.0, 0.0, 0.0, 0]]
Американский Минторг объявил, что внесёт крупнейшую в мире компанию "Хуавэй" и все, связанные с ней, юридические лица в свой черный список. Якобы, это необходимо из соображений нацбезопасности, сообщает телеканал "Россия 24".
['минторг', 'связывать', 'юридический', 'якобы', 'соображение', 'нацбезопасность', 'телеканал']
['якобы', 'соображение', 'телеканал']
[['будто'], [3], ['свой черный список якобы это необходимо из'], [-7.779136398332408], [0], [0.0], [-2.0012394893607526e-36], [-0.0003944916504963203], [0.0], [0]]
[['будто', 3, 'свой черный список якобы это необходимо из', -7.779136398332408, 0, 0.0, -2.0012394893607526e-36, -0.0003944916504963203, 0.0, 0]]
[['понятие'], [3], ['это необходимо из соображений нацбезопасности сообщает телекана

[['округ', 'территория', 'регион', 'район', 'земля', 'площадь', 'область', 'место', 'круг', 'государство'], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3], ['прибрежных водах английского графства девон выловили огромного', 'прибрежных водах английского графства девон выловили огромного', 'прибрежных водах английского графства девон выловили огромного', 'прибрежных водах английского графства девон выловили огромного', 'прибрежных водах английского графства девон выловили огромного', 'прибрежных водах английского графства девон выловили огромного', 'прибрежных водах английского графства девон выловили огромного', 'прибрежных водах английского графства девон выловили огромного', 'прибрежных водах английского графства девон выловили огромного', 'прибрежных водах английского графства девон выловили огромного'], [-9.782916221614693, -9.167750669702814, -9.467473674744923, -8.778843321072586, -7.3949515170189235, -8.94811852399172, -8.188095962474803, -6.94080213002704, -8.848252027865568, -8.220166447811126

Протестующие против строительства «Северного потока — 2» пробрались на строительную площадку на севере Германии, где осуществляется прокладка газопровода EUGAL, и заползли в часть уже проложенного трубопровода.
['протестовать', 'пробираться', 'строительный', 'площадка', 'север', 'осуществляться', 'прокладка', 'газопровод', 'заползать', 'пролагать', 'трубопровод']
['протестовать', 'пробираться']
[['возражать'], [0], ['протестующие против строительства северного потока пробрались'], [-9.411083605160984], [0], [0.07064851196691634], [6.038787104118532e-31], [4.135767876078944e-05], [0.008665342097043924], [1]]
[['возражать', 0, 'протестующие против строительства северного потока пробрались', -9.411083605160984, 0, 0.07064851196691634, 6.038787104118532e-31, 4.135767876078944e-05, 0.008665342097043924, 1]]
[['вмешиваться'], [3], ['строительства северного потока пробрались на строительную площадку'], [-10.347561357955117], [0], [0.08155014893155378], [4.302754615113035e-35], [0.000104634018

[['период', 'срок', 'пора'], [3, 3, 3], ['переснять восьмой заключительный сезон саги петиция с', 'переснять восьмой заключительный сезон саги петиция с', 'переснять восьмой заключительный сезон саги петиция с'], [-8.752374842778872, -8.796493665966455, -7.732422255969726], [0, 0, 0], [0.1517694553836665, 0.009698796755326429, 0.06255728073313287], [-2.824179812099294e-38, 0.0, 0.0], [-5.562216699672717e-06, 0.0, 0.0], [-0.010820519315642, -0.011644457918338668, -0.004533039967351804], [0, 0, 0]]
[['период', 3, 'переснять восьмой заключительный сезон саги петиция с', -8.752374842778872, 0, 0.1517694553836665, -2.824179812099294e-38, -5.562216699672717e-06, -0.010820519315642, 0], ['срок', 3, 'переснять восьмой заключительный сезон саги петиция с', -8.796493665966455, 0, 0.009698796755326429, 0.0, 0.0, -0.011644457918338668, 0], ['пора', 3, 'переснять восьмой заключительный сезон саги петиция с', -7.732422255969726, 0, 0.06255728073313287, 0.0, 0.0, -0.004533039967351804, 0]]
[['легенда

...

['оао', 'размещать', 'бонд', 'облигация', 'финансирование', 'экологический', 'эмитент', 'евро', 'интерфакс', 'банковский']
[]
Минтруд предложил изменить правила расчета прожиточного минимума пенсионера в регионах. На основе этого показателя правительство определяет социальные доплаты, чтобы неработающий пенсионер не оказался за чертой бедности по меркам региона.
['минтруд', 'прожиточный', 'минимум', 'пенсионер', 'показатель', 'доплата', 'неработающий', 'пенсионер', 'бедность']
['минимум', 'пенсионер', 'показатель', 'пенсионер', 'бедность']
[['дно'], [3], ['правила расчета прожиточного минимума пенсионера в регионах'], [-9.33370511859504], [0], [0.0874784406091186], [-2.538062461841396e-32], [-1.7915616815671314], [-0.003302866333761215], [0]]
[['дно', 3, 'правила расчета прожиточного минимума пенсионера в регионах', -9.33370511859504, 0, 0.0874784406091186, -2.538062461841396e-32, -1.7915616815671314, -0.003302866333761215, 0]]
[['старик', 'ветеран'], [3, 3], ['расчета прожиточного мин

[['выражать', 3, 'учитывая что компания раскрывает месячные показатели продаж', -9.097228978671692, 0, 0.05221433417461971, 0.0, 0.0, -0.02076643244501582, 0], ['открывать', 3, 'учитывая что компания раскрывает месячные показатели продаж', -8.24556163363519, 0, 0.2329274316864361, -1.9397488229143403e-38, -3.824027302812283e-06, -0.025474970463648106, 0], ['объявлять', 3, 'учитывая что компания раскрывает месячные показатели продаж', -8.912749200909342, 0, 0.11359955076745575, -1.763408633812213e-38, -3.4763890539579734e-06, -0.0021804937282761637, 0], ['выдавать', 3, 'учитывая что компания раскрывает месячные показатели продаж', -9.181962566510796, 0, 0.15514391490776355, 0.0, 0.0, 0.03747329304832922, 0]]
[['класс', 'высота'], [3, 3], ['компания раскрывает месячные показатели продаж публикация финансовых', 'компания раскрывает месячные показатели продаж публикация финансовых'], [-8.649251056191542, -9.147517831769072], [0, 0], [0.14749859435234092, 0.09375432545456373], [5.2953610788

['ангел', 'совместный']
[['святой'], [2], ['канцлер германии ангела меркель подтвердила что'], [-8.647458619791227], [0], [0.2208309232414355], [0.0], [0.0], [0.040517767553530634], [0]]
[['святой', 2, 'канцлер германии ангела меркель подтвердила что', -8.647458619791227, 0, 0.2208309232414355, 0.0, 0.0, 0.040517767553530634, 0]]
[['общий'], [3], ['в четверг на совместной пресс-конференции с премьером'], [-7.630359263693887], [0], [0.13061233988844811], [-2.4659854274519287e-30], [-4.008110491431694], [-0.022323548448062145], [0]]
[['общий', 3, 'в четверг на совместной пресс-конференции с премьером', -7.630359263693887, 0, 0.13061233988844811, -2.4659854274519287e-30, -4.008110491431694, -0.022323548448062145, 0]]
Если произойдет реальное военное столкновение между Россией и Украиной, то это приведет к "кровавой бойне", заявил украинский генерал Игорь Романенко. По его словам, сейчас попытка Киева взять под контроль территории Донбасса будет бесполезной из-за якобы присутствующих там р

Президент России Владимир Путин допустил возрождение в стране системы распределения выпускников высших учебных заведений, получивших образование за счет бюджета. Оценить эту возможность глава государства предложил Минздраву и правительству в целом.
['возрождение', 'распределение', 'выпускник', 'учебный', 'заведение', 'бюджет', 'минздрав']
['распределение', 'заведение']
[['отделение'], [3], ['в стране системы распределения выпускников высших учебных'], [-9.405271677576536], [0], [-0.012905884294462064], [5.7585395110484454e-36], [3.1634809616321036e-05], [0.03383177981895747], [1]]
[['отделение', 3, 'в стране системы распределения выпускников высших учебных', -9.405271677576536, 0, -0.012905884294462064, 5.7585395110484454e-36, 3.1634809616321036e-05, 0.03383177981895747, 1]]
[['открытие'], [3], ['выпускников высших учебных заведений получивших образование за'], [-9.452607763469095], [0], [-0.02194132237419809], [-2.3795581714245606e-28], [-6.885305921840633], [0.04872140830569027], [0]

[['кончать', 'оканчивать'], [3, 3], ['социальной инфраструктуры планируется завершить к поздней осени', 'социальной инфраструктуры планируется завершить к поздней осени'], [-9.244492782362403, -9.974308897219814], [0, 0], [0.04413535768514222, 0.24918785490718176], [-5.076888288466614e-32, -5.076890935913376e-32], [-2.397870242926686, -2.397875457500305], [-0.004392251821549183, 0.018342896633994306], [0, 0]]
[['кончать', 3, 'социальной инфраструктуры планируется завершить к поздней осени', -9.244492782362403, 0, 0.04413535768514222, -5.076888288466614e-32, -2.397870242926686, -0.004392251821549183, 0], ['оканчивать', 3, 'социальной инфраструктуры планируется завершить к поздней осени', -9.974308897219814, 0, 0.24918785490718176, -5.076890935913376e-32, -2.397875457500305, 0.018342896633994306, 0]]
Президент России Владимир Путин сообщил, что поговорит с мэром Москвы Сергеем Собяниным о ситуации с планируемым вывозом мусора из Москвы на мусорные полигоны в другие регионы.
['мэр', 'план

In [172]:
class_df

,лемма,индекс,контекст,информативность,словарность,близость,вероятность,логарифм_вероятности,контекстуальность,класс
0,лифт,3,оказались заблокированными на подъемнике когда...,-10.858659,0,0.514473,-2.462136e-35,-6.465851e-05,-0.003079,0
1,происшествие,3,верхней части небоскреба инцидент произошел ут...,-10.369867,0,0.584054,4.410660e-38,8.690950e-06,0.021741,0
2,случай,3,верхней части небоскреба инцидент произошел ут...,-7.246549,0,0.192839,-1.111452e-37,-2.190086e-05,-0.012834,0
3,событие,3,верхней части небоскреба инцидент произошел ут...,-8.652756,0,0.342534,-4.727746e-37,-9.316224e-05,0.006443,0
4,сила,3,позволила скинуть лишний вес машины и увеличить,-7.192949,0,0.116806,2.612243e-37,5.144956e-05,0.025251,1
5,добавлять,3,вес машины и увеличить пространство в салоне,-9.185637,0,0.148120,-3.279849e-36,-6.462914e-04,-0.056513,0
6,прибавлять,3,вес машины и увеличить пространство в салоне,-9.352556,0,0.243960,-1.174885e-36,-2.314620e-04,0.004155,0
7,рождение,3,том что виновником возникновения крупнейшего в...,-9.462067,0,0.057538,3.907926e-38,1.042918e-06,-0.008437,1
8,коллектив,3,крупнейшего в истории штата пожара является эн...,-9.974984,0,0.098571,0.000000e+00,0.000000e+00,0.002871,0
9,местный,1,летний уроженец латвии эдвард пургайлис был,-8.792828,0,0.026178,5.077438e-33,3.476391e-07,0.040156,1


In [173]:
class_df.to_csv('class_df.csv',sep='\t', encoding = 'utf-8')

## Штука. Пока не трогать

In [67]:
for token in complex_words:
    if isinstance(token, Complex_word):
        if token.easier: 
            # окна контекста строим для тех сложных слов, для которых есть варианты замен, которые проще, чем слово
            token.make_window(complex_words, window = 5)
            
            context_lem = [c.lexem for c in token.context]
            
            # определение, насколько слово близко с другими словами из контекста
            closeness = np.mean([token.cos_sim(w) for w in token.context if w.lexem!=token.lexem])
            
            print(token.text, token.place, ':', context_lem)
            
            # генерация n-грамм
            text_3grams = [n for n in ngrams(context_lem, 3)]
            
            print(text_3grams)
            
            p_context = context_prob(text_3grams, use_log=True)
                
            print('\nВероятность слова _{0}_ в контексте {1} = {2}, близость к контексту = {3}\n'.format(token.text, context_lem, p_context,closeness))
            
            best_fitness = -1000
            best_sub = None
            
            for sub in token.easier:
                
                # контекст с новым словом
                sub_context = token.context[:token.place]+[sub]+token.context[token.place+1:]
                
                # определение, насколько слово близко с другими словами из контекста
                sub.closeness = np.mean([sub.cos_sim(w) for w in sub_context if w.lexem!=sub.lexem])
            
                sub_context_lem = [t.lexem for t in sub_context]
                
                sub_3grams = [n for n in ngrams(sub_context_lem, 3)]
                
                p_changed_context = context_prob(sub_3grams, use_log=True)
                
                fit='НЕТ'
                if p_changed_context>p_context:
                    fit='ДА'
                    sub.fitness = p_changed_context
                    if sub.fitness > best_fitness:
                        best_fitness = sub.fitness
                        best_sub = sub
                    elif sub.fitness == best_fitness:
                        if best_sub.complexity < sub.complexity:
                            best_fitness = sub.fitness
                            best_sub = sub
                        
                print('\nВероятность замены _{0}_ в контексте {1} = {2}, разница = {3}, используем? - {4}'.format(sub.text, sub_context_lem, p_changed_context, p_context-p_changed_context, fit))
                print('Близость к контексту = {0}, разница с исходным = {1}\n'.format(sub.closeness, closeness-sub.closeness))
            if best_sub:
                print('Лучшее значение вероятности: {0}\n'.format(best_sub.text))
            

изъяли 3 : ['бракованный', 'белорусский', 'лекарство', 'изымать', 'из', 'обращение', 'лекарственный']
[('бракованный', 'белорусский', 'лекарство'), ('белорусский', 'лекарство', 'изымать'), ('лекарство', 'изымать', 'из'), ('изымать', 'из', 'обращение'), ('из', 'обращение', 'лекарственный')]
	 бракованный белорусский лекарство / бракованный белорусский : 0 / 0 ( 1 / 2876543 ) = 3.476395103427969e-07
	 белорусский лекарство изымать / белорусский лекарство : 0 / 0 ( 1 / 2876543 ) = 3.476395103427969e-07
	 лекарство изымать из / лекарство изымать : 0 / 0 ( 1 / 2876543 ) = 3.476395103427969e-07
	 изымать из обращение / изымать из : 26 / 450 ( 27 / 2876993 ) = 9.38479864219343e-06
	 из обращение лекарственный / из обращение : 0 / 158 ( 1 / 2876701 ) = 3.4762041658135483e-07

Вероятность слова _изъяли_ в контексте ['бракованный', 'белорусский', 'лекарство', 'изымать', 'из', 'обращение', 'лекарственный'] = -71.0648734045552, близость к контексту = 0.06318795363628794

	 бракованный белорусский 

поправить вычисление вероятности - в знаменателе число уникальных нграмм, а не униграмм, ну и переписать с условием n>/=1
изменить вычисление вероятности для логарифма
Написать перплексию для получившейся модели - хз как????  
поправить нахождение best_sub - если их больше 1, надо как-то это обрабатывать  
прогнать статистическую модель на текстах и сформировать таким образом корпус положительных замен, которые потом можно подредачить ручками   
заодно при редактировании ручками можно создать еще один файл, в котором редачить, а потом сравнить с первым, чтобы посчитать точность и проч  
потом все-таки посчитать 

In [ ]:
def perplexity(docs):
    docs = np.asarray(docs)
    out_of_dict = []
   
    tmp_sum_docs = 0.0
    N = 0.0
    for doc in log_progress(docs):
        tmp_sum_words = 0.0
        for word in log_progress(words):
            freq = frequencies.get(word, TOL)
            freq_empirical = frequencies_empirical.get(word, 0.0)
            N += freq_empirical
            if freq == TOL:
                out_of_dict.append(word)
            tmp_sum_words += freq_empirical * np.log(freq)
        tmp_sum_docs += tmp_sum_words
       
    return np.exp(-tmp_sum_docs / N), out_of_dict